In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Charger les données
data = pd.read_csv('Mine_Dataset.csv')

# Étape 1: One Hot Encoding pour la colonne 'S'
encoder = OneHotEncoder(sparse=False, categories='auto')
S_encoded = encoder.fit_transform(data[['S']])

# Ajouter les colonnes encodées au dataset
S_encoded_df = pd.DataFrame(S_encoded, columns=[f"S_{int(i*5)}" for i in encoder.categories_[0]])
data = pd.concat([data.drop(columns=['S']), S_encoded_df], axis=1)

# Étape 2: Génération de données synthétiques pour V et H
def generate_synthetic_data(df, n_samples):
    synthetic_data = []
    for _ in range(n_samples):
        sample = df.sample(1, replace=True).iloc[0]
        # Ajouter du bruit gaussien à V et H
        new_V = sample['V'] + np.random.normal(0, 0.01)  # Écart type de 0.01
        new_H = sample['H'] + np.random.normal(0, 0.01)  # Écart type de 0.01
        synthetic_sample = sample.copy()
        synthetic_sample['V'] = np.clip(new_V, 0, 1)  # Limiter aux bornes [0, 1]
        synthetic_sample['H'] = np.clip(new_H, 0, 1)  # Limiter aux bornes [0, 1]
        synthetic_data.append(synthetic_sample.to_dict())
    return pd.DataFrame(synthetic_data)

# Générer 200 nouveaux échantillons
synthetic_data = generate_synthetic_data(data, 200)

# Validation : s'assurer que les colonnes correspondent entre data et synthetic_data
synthetic_data = synthetic_data.reindex(columns=data.columns)

# Étape 3: Équilibrage des classes M
data_combined = pd.concat([data, synthetic_data], ignore_index=True)
balanced_data = pd.DataFrame()

for m_class in data_combined['M'].unique():
    class_subset = data_combined[data_combined['M'] == m_class]
    balanced_subset = resample(class_subset, replace=True, n_samples=200, random_state=42)
    balanced_data = pd.concat([balanced_data, balanced_subset], ignore_index=True)

# Résultat final
balanced_data.reset_index(drop=True, inplace=True)

# Déplacer la colonne 'M' à la dernière position
columns = [col for col in balanced_data.columns if col != 'M']  # Liste des colonnes sauf 'M'
columns.append('M')  # Ajouter 'M' à la fin
balanced_data = balanced_data[columns]  # Réorganiser les colonnes


# Sauvegarde du dataset augmenté
balanced_data.to_csv('Augmented_Mine_Dataset.csv', index=False)


c:\Users\theob\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
